### Fine tune LLMs using LLaMA Factory

- https://github.com/hiyouga/LLaMA-Factory
- Fine tune mistral 7b instruction version

### Game plan
- mistral > fine tune > Docker natural language commands dataset > RAG

- Q: How to delete a docker container.
- A: docker rm -f [container-name]

### Datasets
 - Data on huggingface
- https://huggingface.co/datasets/MattCoddity/dockerNLcommands

- Use Gradio UI
- https://www.gradio.app/

### Setup libraries

In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 23190, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 23190 (delta 135), reused 115 (delta 115), pack-reused 23001 (from 2)
Receiving objects: 100% (23190/23190), 46.49 MiB | 22.23 MiB/s, done.
Resolving deltas: 100% (16891/16891), done.


In [5]:
%cd LLaMA-Factory/

/content/LLaMA-Factory


In [6]:
!pwd

/content/LLaMA-Factory


In [7]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.12.0
    Uninstalling peft-0.12.0:
      Successfully uninstalled peft-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llamafactory 0.9.2 requires peft<=0.12.0,>=0.11.1, but you have peft 0.15.1 which is incompatible.


In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.5 MB/s eta 0:00:00


In [ ]:
#!pip install peft>=0.12.0,<=0.15.1

/bin/bash: line 1: =0.15.1: No such file or directory


In [8]:
!pip install llamafactory

  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.12.0-py3-none-any.whl (296 kB)
  Attempting uninstall: peft
    Found existing installation: peft 0.15.1
    Uninstalling peft-0.15.1:
      Successfully uninstalled peft-0.15.1


In [ ]:
!DISABLE_VERSION_CHECK=1

### Changes in LLaMA Factory code

In [9]:
!GRADIO_SHARE=1 llamafactory-cli webui

2025-04-25 20:30:18.614456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745613018.648537    3036 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745613018.659032    3036 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-25 20:30:18.691372: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://18bd4582a409456fc3.gradio.live

This share link

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `jarvis` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pu

In [11]:
!huggingface-cli upload RickyRubini/my-phi-tuned /content/LLaMA-Factory/saves/Phi-1.5-1.3B/lora/train_2025-04-25-20-31-25

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`huggingface-cli upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.
Start hashing 48 files.
Finished hashing 48 files.
adapter_model.safetensors:   0% 0.00/28.3M [00:00<?, ?B/s]
adapter_model.safetensors:   0% 0.00/28.3M [00:00<?, ?B/s]

optimizer.pt:   0% 0.00/56.9M [00:00<?, ?B/s]


rng_state.pth:   0% 0.00/14.2k [00:00<?, ?B/s]



Upload 14 LFS files:   0% 0/14 [00:00<?, ?it/s]




scaler.pt:   0% 0.00/988 [00:00<?, ?B/s]

scaler.pt: 100% 988/988 [00:00<00:00, 6.50kB/s]

rng_state.pth: 100% 1

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")
base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5")
model = PeftModel.from_pretrained(base_model, "RickyRubini/my-phi-tuned")

# Define input prompt
query_to_llm = "What is the recipe for making crepes?"

# Tokenize and generate
inputs = tokenizer.encode(query_to_llm, return_tensors="pt")
outputs = model.generate(
    inputs,
    max_new_tokens=150
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
outputs

tensor([[24446,   502,   262,  1407, 15544,   329,  1642,   257,  1126, 12272,
            13,   198,   198,    32,    25,   464,  1407, 15544,   329,  1642,
           257,  1126, 12272,   318,   262,  9391,   290,   262,  1429,   286]])